In [ ]:
import pickle
filename = 'finalized_model.pkl'

with open(filename, 'rb') as pickle_file:
    content = pickle.load(pickle_file)

In [ ]:
content.feature_importances_

### Set up
pip install azureml-sdk
pip install azureml-core

#### Reference:
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-existing-model

#### Where to deploy: 
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

## Connect to Azure ML Workspace

In [ ]:

from azureml.core import Workspace
ws = Workspace.get(name='AzureMLWorkspaceName',
               subscription_id='XXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX',
               resource_group='ResourceGroupName'
               )

## Register existing ML model (in pickle format)

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path="finalized_model.pkl",
                          model_name="finalized_model",
                          description="Test register model from external",
                          workspace=ws)

## This will put in score_pbi2.py file

In [ ]:
import os
import pickle
import json
import numpy as np
import pandas as pd

import sklearn
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


# Called when the deployed service starts
def init():
    global model

    # Get the path where the deployed model can be found.
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'))
    # load models
    with open(model_path + '/finalized_model.pkl', 'rb') as handle:
        model = pickle.load(handle)



input_sample = pd.DataFrame(data=[{
    "alcohol": 13.58,
    "malic_acid": 2.58,
    "ash": 2.69,
    "alcalinity_of_ash": 24.5,
    "magnesium": 105,
    "total_phenols": 1.55,
    "flavanoids": 0.84,
    "nonflavanoid_phenols": 0.39,
    "proanthocyanins": 1.54,
    "color_intensity": 8.66,
    "hue": 0.74 ,
    "od280/od315_of_diluted_wines":1.8,
    "proline":750

}])

# This is an integer type sample. Use the data type that reflects the expected result.
output_sample = np.array([0])

# To indicate that we support a variable length of data input,
# set enforce_shape=False
@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        print("input_data....")
        print(data.columns)
        print(type(data))
        result = model.predict(data)
        print("result.....")
        print(result)
    # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

## Define inference configuration

In [ ]:
!pip install inference_schema

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="pbiendpoint")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
#conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("pandas")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
#conda_dep.add_pip_package("keras")
#conda_dep.add_pip_package("gensim")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

for pip_package in ["inference_schema"]:
    myenv.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script="score_pbi2.py",
                                   environment=myenv)

In [ ]:
# Test on local deployment 
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration()
model = Model(ws, name='finalized_model')
service = Model.deploy(ws, 'pbiendpoint2', [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

# If pass then deploy to ML Service

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "pbiendpoint", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

In [ ]:
#Test Calling Service 
X_df = pd.read_csv('test_scoring_nopredict.csv')

import json

input_payload = json.dumps({
    'data': X_df[0:2].values.tolist()
 # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)